In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
#import tensorflow as tf

import psycopg2 as pg
import pandas.io.sql as psql
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
from sqlalchemy import create_engine
from sqlalchemy import inspect
import config

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# Conncet to postgresql dataBase table
protocol = 'postgresql'
username = 'postgres'
password = config.password
host = 'localhost'
port = 5432
database_name = 'Project4_db'

pg_connection = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'

In [3]:
#Create DataFrame from SQL table 
pg_connection = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
conn = pg.connect(pg_connection)
raw_ibm_df = psql.read_sql('SELECT * FROM ibm_employee_data', conn)

# Display all the DataFrame Columns 
pd.options.display.max_columns = None
display(raw_ibm_df)

,age,attrition,businesstravel,dailyrate,department,distancefromhome,education,educationfield,employeecount,employeenumber,environmentsatisfaction,gender,hourlyrate,jobinvolvement,joblevel,jobrole,jobsatisfaction,maritalstatus,monthlyincome,monthlyrate,numcompaniesworked,over18,overtime,percentsalaryhike,performancerating,relationshipsatisfaction,standardhours,stockoptionlevel,totalworkingyears,trainingtimeslastyear,worklifebalance,yearsatcompany,yearsincurrentrole,yearssincelastpromotion,yearswithcurrmanager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,2,Female,94,3,2,Sales Executive,4,Single,5993,19479,8,Y,Yes,11,3,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,3,Male,61,2,2,Research Scientist,2,Married,5130,24907,1,Y,No,23,4,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,4,Male,92,2,1,Laboratory Technician,3,Single,2090,2396,6,Y,Yes,15,3,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,4,Female,56,3,1,Research Scientist,3,Married,2909,23159,1,Y,Yes,11,3,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,1,Male,40,3,1,Laboratory Technician,2,Married,3468,16632,9,Y,No,12,3,4,80,1,6,3,3,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36,No,Travel_Frequently,884,Research & Development,23,2,Medical,1,2061,3,Male,41,4,2,Laboratory Technician,4,Married,2571,12290,4,Y,No,17,3,3,80,1,17,3,3,5,2,0,3
1466,39,No,Travel_Rarely,613,Research & Development,6,1,Medical,1,2062,4,Male,42,2,3,Healthcare Representative,1,Married,9991,21457,4,Y,No,15,3,1,80,1,9,5,3,7,7,1,7
1467,27,No,Travel_Rarely,155,Research & Development,4,3,Life Sciences,1,2064,2,Male,87,4,2,Manufacturing Director,2,Married,6142,5174,1,Y,Yes,20,4,2,80,1,6,0,3,6,2,0,3
1468,49,No,Travel_Frequently,1023,Sales,2,3,Medical,1,2065,4,Male,63,2,2,Sales Executive,2,Married,5390,13243,2,Y,No,14,3,4,80,0,17,3,2,9,6,0,8


In [4]:
# pd.options.display.max_columns = None
# display(raw_ibm_df)

In [5]:
# #  Import and read the charity_data.csv.
# import pandas as pd 
# raw_ibm_df = pd.read_csv('data/WA_Fn-UseC_-HR-Employee-Attrition.csv')
# pd.options.display.max_columns = None
# display(raw_ibm_df)

In [6]:
# checking for null values
raw_ibm_df.isnull().sum()

age                         0
attrition                   0
businesstravel              0
dailyrate                   0
department                  0
distancefromhome            0
education                   0
educationfield              0
employeecount               0
employeenumber              0
environmentsatisfaction     0
gender                      0
hourlyrate                  0
jobinvolvement              0
joblevel                    0
jobrole                     0
jobsatisfaction             0
maritalstatus               0
monthlyincome               0
monthlyrate                 0
numcompaniesworked          0
over18                      0
overtime                    0
percentsalaryhike           0
performancerating           0
relationshipsatisfaction    0
standardhours               0
stockoptionlevel            0
totalworkingyears           0
trainingtimeslastyear       0
worklifebalance             0
yearsatcompany              0
yearsincurrentrole          0
yearssince

In [7]:
# checking number of uniques values per column
raw_ibm_df.nunique()

age                           43
attrition                      2
businesstravel                 3
dailyrate                    886
department                     3
distancefromhome              29
education                      5
educationfield                 6
employeecount                  1
employeenumber              1470
environmentsatisfaction        4
gender                         2
hourlyrate                    71
jobinvolvement                 4
joblevel                       5
jobrole                        9
jobsatisfaction                4
maritalstatus                  3
monthlyincome               1349
monthlyrate                 1427
numcompaniesworked            10
over18                         1
overtime                       2
percentsalaryhike             15
performancerating              2
relationshipsatisfaction       4
standardhours                  1
stockoptionlevel               4
totalworkingyears             40
trainingtimeslastyear          7
worklifeba

In [8]:
raw_ibm_df.describe()

,age,dailyrate,distancefromhome,education,employeecount,environmentsatisfaction,hourlyrate,jobinvolvement,joblevel,jobsatisfaction,monthlyincome,monthlyrate,numcompaniesworked,percentsalaryhike,performancerating,relationshipsatisfaction,standardhours,stockoptionlevel,totalworkingyears,trainingtimeslastyear,worklifebalance,yearsatcompany,yearsincurrentrole,yearssincelastpromotion,yearswithcurrmanager
count,1470.000000,1470.000000,1470.000000,1470.000000,1470.0,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.0,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000
mean,36.923810,802.485714,9.192517,2.912925,1.0,2.721769,65.891156,2.729932,2.063946,2.728571,6502.931293,14313.103401,2.693197,15.209524,3.153741,2.712245,80.0,0.793878,11.279592,2.799320,2.761224,7.008163,4.229252,2.187755,4.123129
std,9.135373,403.509100,8.106864,1.024165,0.0,1.093082,20.329428,0.711561,1.106940,1.102846,4707.956783,7117.786044,2.498009,3.659938,0.360824,1.081209,0.0,0.852077,7.780782,1.289271,0.706476,6.126525,3.623137,3.222430,3.568136
min,18.000000,102.000000,1.000000,1.000000,1.0,1.000000,30.000000,1.000000,1.000000,1.000000,1009.000000,2094.000000,0.000000,11.000000,3.000000,1.000000,80.0,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,30.000000,465.000000,2.000000,2.000000,1.0,2.000000,48.000000,2.000000,1.000000,2.000000,2911.000000,8047.000000,1.000000,12.000000,3.000000,2.000000,80.0,0.000000,6.000000,2.000000,2.000000,3.000000,2.000000,0.000000,2.000000
50%,36.000000,802.000000,7.000000,3.000000,1.0,3.000000,66.000000,3.000000,2.000000,3.000000,4919.000000,14235.500000,2.000000,14.000000,3.000000,3.000000,80.0,1.000000,10.000000,3.000000,3.000000,5.000000,3.000000,1.000000,3.000000
75%,43.000000,1157.000000,14.000000,4.000000,1.0,4.000000,83.750000,3.000000,3.000000,4.000000,8379.000000,20461.500000,4.000000,18.000000,3.000000,4.000000,80.0,1.000000,15.000000,3.000000,3.000000,9.000000,7.000000,3.000000,7.000000
max,60.000000,1499.000000,29.000000,5.000000,1.0,4.000000,100.000000,4.000000,5.000000,4.000000,19999.000000,26999.000000,9.000000,25.000000,4.000000,4.000000,80.0,3.000000,40.000000,6.000000,4.000000,40.000000,18.000000,15.000000,17.000000


In [9]:
# checking data types
raw_ibm_df.dtypes

age                          int64
attrition                   object
businesstravel              object
dailyrate                    int64
department                  object
distancefromhome             int64
education                    int64
educationfield              object
employeecount                int64
employeenumber              object
environmentsatisfaction      int64
gender                      object
hourlyrate                   int64
jobinvolvement               int64
joblevel                     int64
jobrole                     object
jobsatisfaction              int64
maritalstatus               object
monthlyincome                int64
monthlyrate                  int64
numcompaniesworked           int64
over18                      object
overtime                    object
percentsalaryhike            int64
performancerating            int64
relationshipsatisfaction     int64
standardhours                int64
stockoptionlevel             int64
totalworkingyears   

In [10]:
# Pie chart
# Pie chart
labels = raw_ibm_df['Attrition'].nunique
sizes =  raw_ibm_df['Attrition'].count()
#colors
colors = ['#ff9999','#66b3ff','#99ff99','#ffcc99']
#explsion
explode = (0.05,0.05,0.05,0.05)
 
plt.pie(sizes, colors = colors, labels=labels, autopct='%1.1f%%', startangle=90, pctdistance=0.85, explode = explode)
#draw circle
centre_circle = plt.Circle((0,0),0.70,fc='white')
fig = plt.gcf()
fig.gca().add_artist(centre_circle)
# Equal aspect ratio ensures that pie is drawn as a circle
ax1.axis('equal')  
plt.tight_layout()
plt.show()

KeyError: 'Attrition'

In [11]:
# 

raw_ibm_df['Attrition'].value_counts(normalize=True).plot(kind='pie', pctdistance=0.45, autopct="%.0f%%", explode = (0.1, 0), startangle=30, colors = ['dimgrey', 'green'] , fontsize=15, textprops={'color':"firebrick"})

plt.title(("Distribution of Attrition Y/N"),fontsize=16, color = 'firebrick')

plt.xlabel("")
plt.ylabel("");

# save heatmap as .png file
plt.savefig('attrition_pie.png', dpi=300, bbox_inches='tight')

KeyError: 'Attrition'

##### Note: the data set has imbalanced data

In [ ]:
raw_ibm_df.boxplot(column=['Age', 
                            'DistanceFromHome', 
                            'Education', 
                            'EmployeeCount',
                           'EnvironmentSatisfaction', 
                           'HourlyRate',
                           'JobInvolvement', 
                           'JobLevel', 
                           'JobSatisfaction',
                           'NumCompaniesWorked',
                           'PercentSalaryHike', 
                           'PerformanceRating',
                           'RelationshipSatisfaction', 
                           'StandardHours', 
                           'StockOptionLevel',
                           'TotalWorkingYears', 
                           'TrainingTimesLastYear', 
                           'WorkLifeBalance',
                           'YearsAtCompany', 
                           'YearsInCurrentRole', 
                           'YearsSinceLastPromotion',
                           'YearsWithCurrManager'], 
                           figsize=(20, 10), grid=False, rot=90, fontsize=15)


In [ ]:
raw_ibm_df.boxplot(column=['EmployeeNumber', 'DailyRate'], figsize=(20,5), grid=False,rot=90, fontsize=15)

In [ ]:
# The box plot
sns.set(
        style="ticks",                   # The 'ticks' style
        rc={"figure.figsize": (20, 9),      # width = 6, height = 9
            "figure.facecolor": "ivory",  # Figure colour
            "axes.facecolor": "ivory"})  # Axes colour
b1 = sns.boxplot(data = raw_ibm_df[['Age', 
                            'DistanceFromHome', 
                            'Education', 
                            'EmployeeCount',
                           'EnvironmentSatisfaction', 
                           'HourlyRate',
                           'JobInvolvement', 
                           'JobLevel', 
                           'JobSatisfaction',
                           'NumCompaniesWorked',
                           'PercentSalaryHike', 
                           'PerformanceRating',
                           'RelationshipSatisfaction', 
                           'StandardHours', 
                           'StockOptionLevel',
                           'TotalWorkingYears', 
                           'TrainingTimesLastYear', 
                           'WorkLifeBalance',
                           'YearsAtCompany', 
                           'YearsInCurrentRole', 
                           'YearsSinceLastPromotion',
                           'YearsWithCurrManager']]) 
b1 = sns.stripplot(data = raw_ibm_df[['Age', 
                            'DistanceFromHome', 
                            'Education', 
                            'EmployeeCount',
                           'EnvironmentSatisfaction', 
                           'HourlyRate',
                           'JobInvolvement', 
                           'JobLevel', 
                           'JobSatisfaction',
                           'NumCompaniesWorked',
                           'PercentSalaryHike', 
                           'PerformanceRating',
                           'RelationshipSatisfaction', 
                           'StandardHours', 
                           'StockOptionLevel',
                           'TotalWorkingYears', 
                           'TrainingTimesLastYear', 
                           'WorkLifeBalance',
                           'YearsAtCompany', 
                           'YearsInCurrentRole', 
                           'YearsSinceLastPromotion',
                           'YearsWithCurrManager']], color = "black")
plt.setp(b1.get_xticklabels(), rotation=90)

# Set the x axis and font size
b1.set_label(raw_ibm_df, rot = 90)
# Set the plot title with the pval variable and font size
b1.set_title("TITLE", fontsize = 16)
# Remove axis spines
sns.despine(offset = 5, trim = True)

b1.get_figure();

In [ ]:
sns.set(
        style="ticks",                   # The 'ticks' style
        rc={"figure.figsize": (6, 9),      # width = 6, height = 9
            "figure.facecolor": "ivory",  # Figure colour
            "axes.facecolor": "ivory"})  # Axes colour
b2 = sns.boxplot(data = raw_ibm_df[['EmployeeNumber', 'DailyRate']])
b2 = sns.stripplot(data = raw_ibm_df[['EmployeeNumber', 'DailyRate']], color = "black")
b2.get_figure();


In [ ]:
raw_ibm_df.boxplot(column=['MonthlyIncome', 'MonthlyRate'], figsize=(20,5), grid=False,rot=90, fontsize=15)

In [ ]:
raw_ibm_df.corr()

In [ ]:

# Increase the size of the heatmap.
plt.figure(figsize=(20, 12))

# Store heatmap object in a variable.
heatmap = sns.heatmap(raw_ibm_df.corr(), vmin=-1, vmax=1, annot=True, cmap='BrBG')

# Set the range of values to be displayed on the colormap from -1 to 1, 
# and set the annotation to True to display the correlation values on the heatmap.
heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':20}, pad=12)

# save heatmap as .png file
plt.savefig('heatmap.png', dpi=300, bbox_inches='tight')